In [1]:
import pandas as pd
import os

In [2]:
orders = pd.read_csv('../staging_1/orders/orders.csv')
orders

,order_id,customer_id,order_status,order_date,required_date,shipped_date,store_id,staff_id,Extraction_Date_Time,Source
0,1,259,4,2016-01-01,2016-01-03,2016-01-03,1,2,2024-05-03 23:35:33.118721,DB
1,2,1212,4,2016-01-01,2016-01-04,2016-01-03,2,6,2024-05-03 23:35:33.118721,DB
2,3,523,4,2016-01-02,2016-01-05,2016-01-03,2,7,2024-05-03 23:35:33.118721,DB
3,4,175,4,2016-01-03,2016-01-04,2016-01-05,1,3,2024-05-03 23:35:33.118721,DB
4,5,1324,4,2016-01-03,2016-01-06,2016-01-06,2,6,2024-05-03 23:35:33.118721,DB
...,...,...,...,...,...,...,...,...,...,...
1610,1611,6,3,2018-09-06,2018-09-06,2018-09-05,2,7,2024-05-03 23:35:33.118721,DB
1611,1612,3,3,2018-10-21,2018-10-21,2018-10-20,1,3,2024-05-03 23:35:33.118721,DB
1612,1613,1,3,2018-11-18,2018-11-18,2018-11-17,2,6,2024-05-03 23:35:33.118721,DB
1613,1614,135,3,2018-11-28,2018-11-28,2018-11-27,3,8,2024-05-03 23:35:33.118721,DB


In [3]:
orders['late_delivery'] = orders['shipped_date'] > orders['required_date']

In [4]:
orders['shipped_date'] = pd.to_datetime(orders['shipped_date'])
orders['required_date'] = pd.to_datetime(orders['required_date'])
orders['latency_days'] = (orders['shipped_date'] - orders['required_date']).clip(pd.Timedelta(0))
orders.head()

,order_id,customer_id,order_status,order_date,required_date,shipped_date,store_id,staff_id,Extraction_Date_Time,Source,late_delivery,latency_days
0,1,259,4,2016-01-01,2016-01-03,2016-01-03,1,2,2024-05-03 23:35:33.118721,DB,False,0 days
1,2,1212,4,2016-01-01,2016-01-04,2016-01-03,2,6,2024-05-03 23:35:33.118721,DB,False,0 days
2,3,523,4,2016-01-02,2016-01-05,2016-01-03,2,7,2024-05-03 23:35:33.118721,DB,False,0 days
3,4,175,4,2016-01-03,2016-01-04,2016-01-05,1,3,2024-05-03 23:35:33.118721,DB,True,1 days
4,5,1324,4,2016-01-03,2016-01-06,2016-01-06,2,6,2024-05-03 23:35:33.118721,DB,False,0 days


In [5]:
orders['weekend'] = orders['shipped_date'].dt.dayofweek.isin([6, 0])
orders.head()

,order_id,customer_id,order_status,order_date,required_date,shipped_date,store_id,staff_id,Extraction_Date_Time,Source,late_delivery,latency_days,weekend
0,1,259,4,2016-01-01,2016-01-03,2016-01-03,1,2,2024-05-03 23:35:33.118721,DB,False,0 days,True
1,2,1212,4,2016-01-01,2016-01-04,2016-01-03,2,6,2024-05-03 23:35:33.118721,DB,False,0 days,True
2,3,523,4,2016-01-02,2016-01-05,2016-01-03,2,7,2024-05-03 23:35:33.118721,DB,False,0 days,True
3,4,175,4,2016-01-03,2016-01-04,2016-01-05,1,3,2024-05-03 23:35:33.118721,DB,True,1 days,False
4,5,1324,4,2016-01-03,2016-01-06,2016-01-06,2,6,2024-05-03 23:35:33.118721,DB,False,0 days,False


In [6]:
#Reorder columns to be more readable:
desired_column_order = ['order_id', 'customer_id', 'order_status', 'order_date', 'required_date', 'shipped_date', 'late_delivery', 'latency_days', 'weekend','store_id', 'staff_id', 'Extraction_Date_Time', 'Source']
orders = orders.reindex(columns=desired_column_order)
orders.head()

,order_id,customer_id,order_status,order_date,required_date,shipped_date,late_delivery,latency_days,weekend,store_id,staff_id,Extraction_Date_Time,Source
0,1,259,4,2016-01-01,2016-01-03,2016-01-03,False,0 days,True,1,2,2024-05-03 23:35:33.118721,DB
1,2,1212,4,2016-01-01,2016-01-04,2016-01-03,False,0 days,True,2,6,2024-05-03 23:35:33.118721,DB
2,3,523,4,2016-01-02,2016-01-05,2016-01-03,False,0 days,True,2,7,2024-05-03 23:35:33.118721,DB
3,4,175,4,2016-01-03,2016-01-04,2016-01-05,True,1 days,False,1,3,2024-05-03 23:35:33.118721,DB
4,5,1324,4,2016-01-03,2016-01-06,2016-01-06,False,0 days,False,2,6,2024-05-03 23:35:33.118721,DB


In [7]:
file_path = '../Staging_2/orders/orders.csv'

os.makedirs(os.path.dirname(file_path), exist_ok=True)
orders.to_csv(file_path, index = False) 